In [1]:
import pandas as pd
from math import comb
from ax.service.ax_client import AxClient
import time
import sys

sys.path.append('../../../0_helper_functions')
import sdlnano as sdl
import calculation as calc

In [2]:
# Design space size calculation


def calculate_combinations(num_variables, total_sum, step):
    steps = int(total_sum / step)
    return comb(steps + num_variables - 1, num_variables - 1)

# Define parameters
step = 0.05
total_sum = 1

# Calculate combinations for each group
count_group1 = calculate_combinations(10, total_sum, step)  # 3 Solid lipids + 3 Polymers + 3 Liquid lipids + 1 Drug
count_group2 = calculate_combinations(4, total_sum, step)   # 3 Surfactants + 1 Water

# Calculate total combinations
total_combinations = count_group1 * count_group2


print("organic design space size: ", round(count_group1/(1e6),2), 'Million')
print("aqueous design space size: ", round(count_group2/(1e3),2), 'Thousand')
print("design space size: ", round(total_combinations/(1e9),2), 'Billion')


organic design space size:  10.02 Million
aqueous design space size:  1.77 Thousand
design space size:  17.74 Billion


# Check the iteration number

In [3]:
iteration = sdl.get_iteration_number()
print("This is the iteration: #", iteration)

This is the iteration: # 1


# Check the drug

In [4]:
drug = 'ACE'
print("This is the drug:", drug)

This is the drug: ACE


# Optimizer initialization (N/A)

# Generate recommendations

In [5]:
time_start = time.time()

previous_optimizer_file_name = '../iteration_' + str(iteration-1) + '/optimizer/optimizer_load_' + str(iteration-1) + '.json'
# generate recomms
ax_client_init = AxClient.load_from_json_file(previous_optimizer_file_name)
unlabeled_trial, ax_client_design = sdl.generate_trials(ax_client=ax_client_init, num_of_trials = 16, drug = drug, bopt=1)

time_end = time.time()

print(round((time_end-time_start)/60), 'min')

/Users/zeqing/opt/anaconda3/envs/sdlnano_plot/lib/python3.11/site-packages/botorch/acquisition/multi_objective/logei.py:418: RuntimeWarning: `cache_root` is only supported for GPyTorchModels that are not MultiTask models and don't produce a TransformedPosterior. Got a model of type <class 'botorch.models.model.ModelList'>. Setting `cache_root = False`.
  NoisyExpectedHypervolumeMixin.__init__(
[INFO 08-02 13:21:35] ax.service.ax_client: Generated new trial 16 with parameters {'Drug': 40, 'SL_1': 0, 'SL_2': 61, 'SL_3': 0, 'LL_1': 0, 'LL_2': 60, 'LL_3': 75, 'P_1': 56, 'P_2': 36, 'P_3': 0, 'S_1': 45, 'S_2': 85, 'S_3': 28, 'Water': 0} using model BoTorch.
/Users/zeqing/opt/anaconda3/envs/sdlnano_plot/lib/python3.11/site-packages/ax/core/data.py:293: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude 

14 min


In [12]:
# process the trails into ratios

unlabeled_trial_processed = sdl.process_trails(unlabeled_trial)
unlabeled_trial_processed.describe()

,trial_index,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,P_3,S_1,S_2,S_3,Water
count,16.000000,16.000000,16.0,16.000000,16.0,16.000000,16.000000,16.000000,16.000000,16.000000,16.00000,16.000000,16.000000,16.000000,16.000
mean,23.500000,0.181812,0.0,0.179000,0.0,0.007562,0.199187,0.267625,0.122937,0.030125,0.01175,0.154438,0.768937,0.034625,0.042
std,4.760952,0.096788,0.0,0.131924,0.0,0.030250,0.046355,0.110340,0.117405,0.072688,0.04700,0.218248,0.278818,0.101415,0.168
min,16.000000,0.004000,0.0,0.000000,0.0,0.000000,0.098000,0.094000,0.000000,0.000000,0.00000,0.000000,0.328000,0.000000,0.000
25%,19.750000,0.117750,0.0,0.105750,0.0,0.000000,0.179500,0.179500,0.000000,0.000000,0.00000,0.000000,0.513000,0.000000,0.000
50%,23.500000,0.192500,0.0,0.177500,0.0,0.000000,0.194500,0.249500,0.138500,0.000000,0.00000,0.000000,1.000000,0.000000,0.000
75%,27.250000,0.231500,0.0,0.252500,0.0,0.000000,0.229250,0.332250,0.193500,0.000000,0.00000,0.305250,1.000000,0.000000,0.000
max,31.000000,0.348000,0.0,0.446000,0.0,0.121000,0.289000,0.475000,0.356000,0.265000,0.18800,0.549000,1.000000,0.377000,0.672


In [13]:
unlabeled_trial
unlabeled_trial.describe()

,trial_index,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,P_3,S_1,S_2,S_3,Water
count,16.000000,16.000000,16.0,16.000000,16.0,16.0,16.00000,16.000000,16.00000,16.000000,16.00,16.000000,16.000000,16.000000,16.0
mean,23.500000,51.250000,0.0,55.500000,0.0,4.0,56.00000,72.250000,38.56250,12.000000,6.25,26.812500,76.687500,7.562500,5.0
std,4.760952,24.591326,0.0,38.187258,0.0,16.0,9.93311,18.053624,35.60144,28.333725,25.00,38.377456,19.327765,23.829866,20.0
min,16.000000,1.000000,0.0,0.000000,0.0,0.0,36.00000,40.000000,0.00000,0.000000,0.00,0.000000,33.000000,0.000000,0.0
25%,19.750000,39.750000,0.0,27.000000,0.0,0.0,51.25000,57.500000,0.00000,0.000000,0.00,0.000000,80.750000,0.000000,0.0
50%,23.500000,50.000000,0.0,65.500000,0.0,0.0,55.00000,75.000000,42.00000,0.000000,0.00,0.000000,85.500000,0.000000,0.0
75%,27.250000,62.500000,0.0,84.000000,0.0,0.0,60.75000,86.000000,61.50000,0.000000,0.00,52.250000,87.500000,0.000000,0.0
max,31.000000,100.000000,0.0,100.000000,0.0,64.0,78.00000,100.000000,100.00000,99.000000,100.00,100.000000,89.000000,93.000000,80.0


In [14]:
# Convert ratios to volumes

transfer = calc.converter(unlabeled_trial_processed)
transfer.describe()

,trial_index,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,P_3,S_1,S_2,S_3,Water,Solvent
count,16.000000,16.000000,16.0,16.0000,16.0,16.000000,16.000000,16.000000,16.000000,16.000000,16.0000,16.000000,16.000000,16.000000,16.0,16.000000
mean,23.500000,63.634375,0.0,62.6500,0.0,2.646875,69.715625,93.668750,43.028125,10.543750,4.1125,154.437500,768.937500,34.625000,42.0,0.000000
std,4.760952,33.875762,0.0,46.1733,0.0,10.587500,16.224413,38.619002,41.091743,25.440891,16.4500,218.247556,278.817854,101.415236,168.0,0.255604
min,16.000000,1.400000,0.0,0.0000,0.0,0.000000,34.300000,32.900000,0.000000,0.000000,0.0000,0.000000,328.000000,0.000000,0.0,-0.350000
25%,19.750000,41.212500,0.0,37.0125,0.0,0.000000,62.825000,62.825000,0.000000,0.000000,0.0000,0.000000,513.000000,0.000000,0.0,-0.087500
50%,23.500000,67.375000,0.0,62.1250,0.0,0.000000,68.075000,87.325000,48.475000,0.000000,0.0000,0.000000,1000.000000,0.000000,0.0,0.000000
75%,27.250000,81.025000,0.0,88.3750,0.0,0.000000,80.237500,116.287500,67.725000,0.000000,0.0000,305.250000,1000.000000,0.000000,0.0,0.087500
max,31.000000,121.800000,0.0,156.1000,0.0,42.350000,101.150000,166.250000,124.600000,92.750000,65.8000,549.000000,1000.000000,377.000000,672.0,0.350000


In [15]:
#dead_volume = 3.0 # mL

round(transfer[['Drug','SL_1', 'SL_2', 'SL_3', 'LL_1', 'LL_2', 'LL_3', 'P_1', 'P_2', 'P_3', 'S_1', 'S_2', 'S_3', 'Water', 'Solvent']].sum()/1000,1)

Drug        1.0
SL_1        0.0
SL_2        1.0
SL_3        0.0
LL_1        0.0
LL_2        1.1
LL_3        1.5
P_1         0.7
P_2         0.2
P_3         0.1
S_1         2.5
S_2        12.3
S_3         0.6
Water       0.7
Solvent     0.0
dtype: float64

In [16]:
# file path
ax_client_design_path = "optimizer/optimizer_design_" + str(iteration) + ".json"

unlabeled_trial_path = "data/unlabeled_" + str(iteration) + ".xlsx"

unlabeled_trial_processed_path = "data/unlabeled_processed_" + str(iteration) + ".xlsx"

transfer_path = 'data/transfer_' + str(iteration) + '.xlsx'


In [17]:
ax_client_design.save_to_json_file(ax_client_design_path)

unlabeled_trial.to_excel(unlabeled_trial_path, index=False)

unlabeled_trial_processed.to_excel(unlabeled_trial_processed_path, index=False)

transfer.to_excel(transfer_path, index = False)

# Generate new protocol

In [18]:
# Paths
template_protocol = '../../templates/OT_protocol_template.py'

output_script_path_1 = 'protocol/iteration_' + str(iteration) + '_OT_2_protocol_1.py'
output_script_path_2 = 'protocol/iteration_' + str(iteration) + '_OT_2_protocol_2.py'



In [19]:
# Update the script with Excel data
updated_script = calc.update_transfer_script(template_protocol, transfer_path, output_script_path_1, output_script_path_2) 


In [20]:
updated_script

('protocol/iteration_1_OT_2_protocol_1.py',
 'protocol/iteration_1_OT_2_protocol_2.py')

# Perform experiment

# Results

In [21]:
results = calc.parameters_and_complexity(iteration)
results

,trial_index,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,P_3,S_1,S_2,S_3,Water,Complexity,Complexity_STD
0,16,0.122,0,0.186,0,0.000,0.183,0.229,0.171,0.110,0.000,0.285,0.538,0.177,0.000,8,0
1,17,0.147,0,0.239,0,0.000,0.191,0.232,0.191,0.000,0.000,0.366,0.634,0.000,0.000,6,0
2,18,0.180,0,0.252,0,0.000,0.187,0.180,0.201,0.000,0.000,0.000,1.000,0.000,0.000,5,0
3,19,0.141,0,0.141,0,0.000,0.237,0.330,0.152,0.000,0.000,0.000,1.000,0.000,0.000,5,0
4,20,0.205,0,0.281,0,0.000,0.188,0.178,0.147,0.000,0.000,0.000,1.000,0.000,0.000,5,0
5,21,0.212,0,0.000,0,0.000,0.233,0.267,0.288,0.000,0.000,0.000,1.000,0.000,0.000,4,0
6,22,0.105,0,0.155,0,0.000,0.169,0.231,0.075,0.265,0.000,0.263,0.360,0.377,0.000,8,0
7,23,0.092,0,0.169,0,0.121,0.098,0.094,0.130,0.107,0.188,0.000,0.328,0.000,0.672,8,0
8,24,0.044,0,0.000,0,0.000,0.289,0.311,0.356,0.000,0.000,0.000,1.000,0.000,0.000,4,0
9,25,0.310,0,0.145,0,0.000,0.198,0.347,0.000,0.000,0.000,0.470,0.530,0.000,0.000,5,0


In [22]:
size_raw = calc.size_raw(iteration)
size_raw

,Data Quality,Item,Size,PD Index
0,Good,A1,280.907963,0.203723
1,Good,A2,298.516069,0.259255
2,Good,A3,291.712160,0.283113
3,Good,A4,315.116535,0.229564
4,Good,A5,287.988598,0.266512
5,Good,A6,304.129340,0.239702
6,Caution,B1,280.101124,0.212337
7,Caution,B2,281.589117,0.197322
8,Caution,B3,277.711361,0.180883
9,Caution,B4,262.829797,0.150971


In [23]:
size_processed = calc.process_formulations(size_raw)
size_processed

,Formulation,Size,Size_STD,PDI,PDI_STD,Formulation Quality
0,Formulation 1,290.378731,8.879463,0.248697,0.040735,1
1,Formulation 2,302.411491,13.645310,0.245259,0.019091,1
2,Formulation 3,1000.000000,0.000000,1.000000,0.000000,0
3,Formulation 4,1000.000000,0.000000,1.000000,0.000000,0
4,Formulation 5,294.768351,9.271564,0.251449,0.051208,1
5,Formulation 6,1000.000000,0.000000,1.000000,0.000000,0
6,Formulation 7,273.123986,9.115731,0.279973,0.017372,1
7,Formulation 8,212.275950,6.783976,0.211617,0.013560,1
8,Formulation 9,1000.000000,0.000000,1.000000,0.000000,0
9,Formulation 10,1000.000000,0.000000,1.000000,0.000000,0


In [24]:
results_final = pd.concat([results, size_processed], axis=1)
results_final

,trial_index,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,...,S_3,Water,Complexity,Complexity_STD,Formulation,Size,Size_STD,PDI,PDI_STD,Formulation Quality
0,16,0.122,0,0.186,0,0.000,0.183,0.229,0.171,0.110,...,0.177,0.000,8,0,Formulation 1,290.378731,8.879463,0.248697,0.040735,1
1,17,0.147,0,0.239,0,0.000,0.191,0.232,0.191,0.000,...,0.000,0.000,6,0,Formulation 2,302.411491,13.645310,0.245259,0.019091,1
2,18,0.180,0,0.252,0,0.000,0.187,0.180,0.201,0.000,...,0.000,0.000,5,0,Formulation 3,1000.000000,0.000000,1.000000,0.000000,0
3,19,0.141,0,0.141,0,0.000,0.237,0.330,0.152,0.000,...,0.000,0.000,5,0,Formulation 4,1000.000000,0.000000,1.000000,0.000000,0
4,20,0.205,0,0.281,0,0.000,0.188,0.178,0.147,0.000,...,0.000,0.000,5,0,Formulation 5,294.768351,9.271564,0.251449,0.051208,1
5,21,0.212,0,0.000,0,0.000,0.233,0.267,0.288,0.000,...,0.000,0.000,4,0,Formulation 6,1000.000000,0.000000,1.000000,0.000000,0
6,22,0.105,0,0.155,0,0.000,0.169,0.231,0.075,0.265,...,0.377,0.000,8,0,Formulation 7,273.123986,9.115731,0.279973,0.017372,1
7,23,0.092,0,0.169,0,0.121,0.098,0.094,0.130,0.107,...,0.000,0.672,8,0,Formulation 8,212.275950,6.783976,0.211617,0.013560,1
8,24,0.044,0,0.000,0,0.000,0.289,0.311,0.356,0.000,...,0.000,0.000,4,0,Formulation 9,1000.000000,0.000000,1.000000,0.000000,0
9,25,0.310,0,0.145,0,0.000,0.198,0.347,0.000,0.000,...,0.000,0.000,5,0,Formulation 10,1000.000000,0.000000,1.000000,0.000000,0


In [25]:
# µg/mL
results_final['Solu'] = results_final['Drug'] * 600 / 0.3
results_final['Solu_STD'] = 0

results_final.loc[results_final['Formulation Quality'] == 0, 'Solu'] = 0
results_final

,trial_index,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,...,Complexity,Complexity_STD,Formulation,Size,Size_STD,PDI,PDI_STD,Formulation Quality,Solu,Solu_STD
0,16,0.122,0,0.186,0,0.000,0.183,0.229,0.171,0.110,...,8,0,Formulation 1,290.378731,8.879463,0.248697,0.040735,1,244.0,0
1,17,0.147,0,0.239,0,0.000,0.191,0.232,0.191,0.000,...,6,0,Formulation 2,302.411491,13.645310,0.245259,0.019091,1,294.0,0
2,18,0.180,0,0.252,0,0.000,0.187,0.180,0.201,0.000,...,5,0,Formulation 3,1000.000000,0.000000,1.000000,0.000000,0,0.0,0
3,19,0.141,0,0.141,0,0.000,0.237,0.330,0.152,0.000,...,5,0,Formulation 4,1000.000000,0.000000,1.000000,0.000000,0,0.0,0
4,20,0.205,0,0.281,0,0.000,0.188,0.178,0.147,0.000,...,5,0,Formulation 5,294.768351,9.271564,0.251449,0.051208,1,410.0,0
5,21,0.212,0,0.000,0,0.000,0.233,0.267,0.288,0.000,...,4,0,Formulation 6,1000.000000,0.000000,1.000000,0.000000,0,0.0,0
6,22,0.105,0,0.155,0,0.000,0.169,0.231,0.075,0.265,...,8,0,Formulation 7,273.123986,9.115731,0.279973,0.017372,1,210.0,0
7,23,0.092,0,0.169,0,0.121,0.098,0.094,0.130,0.107,...,8,0,Formulation 8,212.275950,6.783976,0.211617,0.013560,1,184.0,0
8,24,0.044,0,0.000,0,0.000,0.289,0.311,0.356,0.000,...,4,0,Formulation 9,1000.000000,0.000000,1.000000,0.000000,0,0.0,0
9,25,0.310,0,0.145,0,0.000,0.198,0.347,0.000,0.000,...,5,0,Formulation 10,1000.000000,0.000000,1.000000,0.000000,0,0.0,0


In [26]:
results_normalized = sdl.normalize(results_final)
results_normalized[['Formulation Quality','Size', 'Size_STD', 'Solu', 'Solu_STD', 'PDI', 'PDI_STD','Complexity']]

,Formulation Quality,Size,Size_STD,Solu,Solu_STD,PDI,PDI_STD,Complexity
0,1,0.290379,0.008879,0.122,0.0,0.248697,0.040735,0.666667
1,1,0.302411,0.013645,0.147,0.0,0.245259,0.019091,0.500000
2,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.416667
3,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.416667
4,1,0.294768,0.009272,0.205,0.0,0.251449,0.051208,0.416667
5,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.333333
6,1,0.273124,0.009116,0.105,0.0,0.279973,0.017372,0.666667
7,1,0.212276,0.006784,0.092,0.0,0.211617,0.013560,0.666667
8,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.333333
9,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.416667


In [27]:
results_normalized = sdl.normalize(results_final)
results_normalized[['Formulation Quality','Size', 'Size_STD', 'Solu', 'Solu_STD', 'PDI', 'PDI_STD','Complexity']]

,Formulation Quality,Size,Size_STD,Solu,Solu_STD,PDI,PDI_STD,Complexity
0,1,0.290379,0.008879,0.122,0.0,0.248697,0.040735,0.666667
1,1,0.302411,0.013645,0.147,0.0,0.245259,0.019091,0.500000
2,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.416667
3,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.416667
4,1,0.294768,0.009272,0.205,0.0,0.251449,0.051208,0.416667
5,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.333333
6,1,0.273124,0.009116,0.105,0.0,0.279973,0.017372,0.666667
7,1,0.212276,0.006784,0.092,0.0,0.211617,0.013560,0.666667
8,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.333333
9,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.416667


In [28]:
labeled_data_path = 'data/labeled_' + str(iteration) + '.xlsx'
labeled_norm_path = 'data/labeled_norm_' + str(iteration) + '.xlsx'


In [29]:
results_final.to_excel(labeled_data_path, index=False)
results_normalized.to_excel(labeled_norm_path, index=False)

# Load the labeled data to update the optimizer

In [30]:
labeled_norm = pd.read_excel(labeled_norm_path)
labeled_norm.head()

,trial_index,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,...,Complexity,Complexity_STD,Formulation,Size,Size_STD,PDI,PDI_STD,Formulation Quality,Solu,Solu_STD
0,16,0.122,0,0.186,0,0.0,0.183,0.229,0.171,0.11,...,0.666667,0,Formulation 1,0.290379,0.008879,0.248697,0.040735,1,0.122,0
1,17,0.147,0,0.239,0,0.0,0.191,0.232,0.191,0.00,...,0.500000,0,Formulation 2,0.302411,0.013645,0.245259,0.019091,1,0.147,0
2,18,0.180,0,0.252,0,0.0,0.187,0.180,0.201,0.00,...,0.416667,0,Formulation 3,1.000000,0.000000,1.000000,0.000000,0,0.000,0
3,19,0.141,0,0.141,0,0.0,0.237,0.330,0.152,0.00,...,0.416667,0,Formulation 4,1.000000,0.000000,1.000000,0.000000,0,0.000,0
4,20,0.205,0,0.281,0,0.0,0.188,0.178,0.147,0.00,...,0.416667,0,Formulation 5,0.294768,0.009272,0.251449,0.051208,1,0.205,0


In [31]:
ax_client_design = AxClient.load_from_json_file(ax_client_design_path)


In [32]:
loaded_optimizer_path = "optimizer/optimizer_load_" + str(iteration) + ".json"

In [33]:
loaded_ax_client = sdl.load_labeled_data(ax_client_design, labeled_norm_path)
loaded_ax_client.save_to_json_file(loaded_optimizer_path)

[INFO 08-02 15:27:14] ax.service.ax_client: Completed trial 16 with data: {'Solu': (0.122, 0), 'Size': (0.290379, 0.008879), 'PDI': (0.248697, 0.040735)}.
[INFO 08-02 15:27:14] ax.service.ax_client: Completed trial 17 with data: {'Solu': (0.147, 0), 'Size': (0.302411, 0.013645), 'PDI': (0.245259, 0.019091)}.
[INFO 08-02 15:27:14] ax.service.ax_client: Completed trial 18 with data: {'Solu': (0.0, 0), 'Size': (1.0, 0.0), 'PDI': (1.0, 0.0)}.
[INFO 08-02 15:27:14] ax.service.ax_client: Completed trial 19 with data: {'Solu': (0.0, 0), 'Size': (1.0, 0.0), 'PDI': (1.0, 0.0)}.
[INFO 08-02 15:27:14] ax.service.ax_client: Completed trial 20 with data: {'Solu': (0.205, 0), 'Size': (0.294768, 0.009272), 'PDI': (0.251449, 0.051208)}.
[INFO 08-02 15:27:14] ax.service.ax_client: Completed trial 21 with data: {'Solu': (0.0, 0), 'Size': (1.0, 0.0), 'PDI': (1.0, 0.0)}.
[INFO 08-02 15:27:14] ax.service.ax_client: Completed trial 22 with data: {'Solu': (0.105, 0), 'Size': (0.273124, 0.009116), 'PDI': (0.2